In [2]:
import jupyterthemes as jt

In [15]:
!jt -l

Available Themes: 
   chesterish
   grade3
   gruvboxd
   gruvboxl
   monokai
   oceans16
   onedork
   solarizedd
   solarizedl


In [18]:
!jt -t onedork

# Building the Model

In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
from keras import applications
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
import math
import os
import cv2

In [2]:
df_train = pd.read_csv('./train.csv')
df_train.head()

,Image,Class
0,image3476.jpg,Miscellaneous
1,image5198.jpg,Candle
2,image4183.jpg,Snowman
3,image1806.jpg,Miscellaneous
4,image7831.jpg,Miscellaneous


In [3]:
# number of training classes
df_train['Class'].value_counts()

Miscellaneous     2801
Christmas_Tree    1539
Jacket             640
Candle             593
Airplane           535
Snowman            361
Name: Class, dtype: int64

In [4]:
# training data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
         shear_range=0.2,
         zoom_range=0.2,
         horizontal_flip=True,
         width_shift_range=0.2,
         height_shift_range=0.2,
         fill_mode = 'nearest')

train_generator=train_datagen.flow_from_dataframe(dataframe=df_train,
                                            directory="./train/",
                                            x_col="Image",
                                            y_col="Class",
                                            batch_size=128,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(299,299))

Found 6469 validated image filenames belonging to 6 classes.


In [29]:
import tensorflow as tf
#loading pre-trained models
res = tf.keras.applications.InceptionResNetV2(
    include_top=True,
    weights="imagenet"
)

In [30]:
x = res.layers[-2].output
fc = Dense(2048,activation='relu')(x)
fc = Dense(1024,activation='relu')(fc)
fc = Dense(512,activation='relu')(fc)
fc = Dense(256,activation='relu')(fc)
fc = Dense(6,activation='softmax')(fc)

In [31]:
# created final models
model = Model(inputs=res.input,outputs=fc)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 149, 149, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 149, 149, 32) 0           batch_normalization_94[0][0]     
____________________________________________________________________________________________

block8_6_ac (Activation)        (None, 8, 8, 2080)   0           block8_6[0][0]                   
__________________________________________________________________________________________________
conv2d_282 (Conv2D)             (None, 8, 8, 192)    399360      block8_6_ac[0][0]                
__________________________________________________________________________________________________
batch_normalization_282 (BatchN (None, 8, 8, 192)    576         conv2d_282[0][0]                 
__________________________________________________________________________________________________
activation_282 (Activation)     (None, 8, 8, 192)    0           batch_normalization_282[0][0]    
__________________________________________________________________________________________________
conv2d_283 (Conv2D)             (None, 8, 8, 224)    129024      activation_282[0][0]             
__________________________________________________________________________________________________
batch_norm

In [32]:
#freezing all the layers
for layer in model.layers[:-16]:
    #print(l)
    layer.trainable = False

In [33]:
#compiling model
model.compile(optimizer='adam',loss ="categorical_crossentropy",metrics=["accuracy"])

In [34]:
history = model.fit(train_generator, epochs=5)

Epoch 1/5
51/51 [==============================] - 934s 18s/step - loss: 0.9966 - accuracy: 0.6254
Epoch 2/5
51/51 [==============================] - 941s 18s/step - loss: 0.3237 - accuracy: 0.8820
Epoch 3/5
51/51 [==============================] - 931s 18s/step - loss: 0.3028 - accuracy: 0.8895
Epoch 4/5
51/51 [==============================] - 931s 18s/step - loss: 0.2600 - accuracy: 0.9111
Epoch 5/5
51/51 [==============================] - 932s 18s/step - loss: 0.2436 - accuracy: 0.9096


In [28]:
model.save('./Models/Inception/Model2.h5')

In [23]:
classes = train_generator.class_indices

In [24]:
classes = dict((v,k) for k,v in classes.items())
classes = [classes[k] for k in classes]

In [25]:
from keras.preprocessing import image
Image = []
prediction = []


for i, file in enumerate(os.listdir('./test/')):
    Image.append(file)
    file = './test/' + file

    img = image.load_img(file, target_size=(299,299,3)) 
    img = image.img_to_array(img)
    img = img/255
    pred = model.predict(img.reshape(1,299,299,3))

    prediction.append(classes[np.argmax(pred[0])])

In [26]:
# submission file creation
data=pd.DataFrame((zip(Image, prediction)),columns=['Image','Class'])
data.head()

,Image,Class
0,image10.jpg,Miscellaneous
1,image100.jpg,Airplane
2,image1013.jpg,Jacket
3,image1014.jpg,Miscellaneous
4,image1018.jpg,Christmas_Tree


In [27]:
data.to_csv('./result_inception.csv',index=False)